In [ ]:
import pandas as pd

In [150]:
consultas = pd.read_excel('/content/Relatorio_Slip_Fatura-816125-201020251329.xls', index_col=False)
dados = pd.read_excel('/content/DADOS PARA A DIRF - trabalhando.xlsx', sheet_name=('JUL 2025'), index_col=False, skiprows=1)

In [130]:
dados.head()

,CÓD DA FAMILIA,CARTEIRA,DEPENDENCIA,NOME,CPF,NASCIMENTO,MENSALIDADE,CONSULTA,RETROATIVO RN,RETROATIVO,TOTAL
0,90.0,1.371384e+15,Titular,ADAILTON SANTOS OLIVEIRA,738.717.296-04,06/03/1969,323.89,NaN,NaN,NaN,323.89
1,90.0,1.371384e+15,Esposa,APARECIDA RAIMUNDA DIAS OLIVEIRA,004.080.536-07,15/09/1970,323.89,NaN,NaN,NaN,323.89
2,90.0,1.371384e+15,Filho,ANDRE AYRES DIAS OLIVEIRA,131.684.916-32,30/09/2002,323.89,NaN,NaN,NaN,323.89
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,251.0,1.371384e+15,Titular,ADAIR SALVADO JUNIOR,430.541.966-15,13/01/1962,743.94,NaN,NaN,NaN,743.94


In [142]:
consultas.head()

,Evento,Unnamed: 1,Gp. Ap.,Doc. Finan.,Contrato Financeiro
0,Prestador,Qtde,Dt. Real.,Servico,Total Cobr.
1,Cód Titular: 1371384000345009 - ADLER DINIZ DE...,NaN,NaN,NaN,NaN
2,Beneficiário: 1371384000345009 - ADLER DINIZ D...,NaN,NaN,NaN,NaN
3,8121814,NaN,CONSULTAS,752225,137138400
4,MARCELO MENI GONCALVES,"1,0000",29/04/2025 14:13:51,10101012 - Consulta em consultório (no horário...,59.95


In [151]:
beneficiarios_df = consultas[consultas['Evento'].str.startswith('Beneficiário:', na=False)].copy()

results = []

beneficiario_name = None
for index, row in consultas.iterrows():
    if row['Evento'] is not None and isinstance(row['Evento'], str) and row['Evento'].startswith('Beneficiário:'):

        beneficiario_name = row['Evento'].split(' - ')[1].split(' (')[0]
    elif row['Doc. Finan.'] == 'Total Crédito' and beneficiario_name is not None:

        total_credito = row['Contrato Financeiro']
        results.append({'Nome Beneficiário': beneficiario_name, 'Total Crédito': total_credito})
        beneficiario_name = None


beneficiarios = pd.DataFrame(results)

In [152]:
display(beneficiarios)

,Nome Beneficiário,Total Crédito
0,ADAIR SALVADO JUNIOR,119.90
1,ADLER DINIZ DE SOUZA,59.95
2,KATIUCIA DIAS FERNANDES,59.95
3,SEBASTIAO FRANCISCO DE SOUZA,119.90
4,TAINA FERNANDES DINIZ,59.95
...,...,...
422,WELBERSOM SILVA VIANA,59.95
423,WELLINGTON LAGE LOPES,59.95
424,WLAMIR LEANDRO DE OLIVEIRA GIFFONI,59.95
425,YONE DOMINGUES DOS SANTOS,179.85


In [153]:
beneficiarios['Total Crédito'].sum()

np.float64(40825.95000000001)

In [154]:
for index, row in beneficiarios.iterrows():
    beneficiario_name = row['Nome Beneficiário']
    total_credito = row['Total Crédito']

    dados.loc[dados['NOME'] == beneficiario_name, 'CONSULTA'] = total_credito

display(dados.head())

,CÓD DA FAMILIA,CARTEIRA,DEPENDENCIA,NOME,CPF,NASCIMENTO,MENSALIDADE,CONSULTA,RETROATIVO RN,RETROATIVO,TOTAL
0,90.0,1.371384e+15,Titular,ADAILTON SANTOS OLIVEIRA,738.717.296-04,06/03/1969,323.89,NaN,NaN,NaN,323.89
1,90.0,1.371384e+15,Esposa,APARECIDA RAIMUNDA DIAS OLIVEIRA,004.080.536-07,15/09/1970,323.89,NaN,NaN,NaN,323.89
2,90.0,1.371384e+15,Filho,ANDRE AYRES DIAS OLIVEIRA,131.684.916-32,30/09/2002,323.89,NaN,NaN,NaN,323.89
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,251.0,1.371384e+15,Titular,ADAIR SALVADO JUNIOR,430.541.966-15,13/01/1962,743.94,119.9,NaN,NaN,743.94


In [155]:
dados.to_excel('consultas.xlsx', index=0)

In [156]:
dados_novo = dados[:-33]
duplicate_names = dados_novo[dados_novo['CONSULTA'].notna() & dados_novo['NOME'].duplicated(keep=False)]

if not duplicate_names.empty:
    print("Nomes duplicados encontrados na planilha 'dados' (excluindo NaN):")
    display(duplicate_names)
else:
    print("Não há nomes duplicados na planilha 'dados' (excluindo NaN).")

Não há nomes duplicados na planilha 'dados' (excluindo NaN).
